In [1]:
import boto3
import src.roc._manual_labels_config as roc_manual_labels_config
import src.roc.state as roc_state
from pathlib import Path 
import numpy as np 
from src.aau.S3Manager import S3Manager
from src.ROC_Classifier.DataPipe import S3ROCManager
from src.utils.PathManager import Paths as RepoPath
import pandas as pd 
import yaml 
config = roc_manual_labels_config.init()

In [2]:
S3 = S3Manager(config['procdata_info'])

In [3]:
S3.list_files('ROC/PROCESSED_DATA', file_prefix="POND1_ROC_PROCESSED_DATA")

['ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20160601_20160701.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20160701_20160801.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20160801_20160901.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20160901_20161001.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20161001_20161101.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20161101_20161201.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20161201_20170101.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20170101_20170201.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20170201_20170301.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20170301_20170401.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20170401_20170501.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20170501_20170601.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20170601_20170701.csv',
 'ROC/PROCESSED_DATA/POND1_ROC_PROCESSED_DATA_20170701_20170801.csv',
 'ROC/PROCESSED_DATA

In [4]:
df = S3.read_from_storage(path='ROC/SOLAR_DATA', file_prefix='SOLAR_DATA', item_cd = "Daralingie", start='2018-03-01', end = '2018-11-01')
df.head()

,time,temperature_2m,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,TS
0,2018-03-01T00:00,28.4,0.0,0.0,0.0,0.0,632.0,533.0,99.0,866.0,2018-03-01 00:00:00
1,2018-03-01T01:00,31.6,0.0,0.0,0.0,0.0,822.0,711.0,111.0,922.6,2018-03-01 01:00:00
2,2018-03-01T02:00,32.7,0.0,0.0,0.0,0.0,955.0,836.0,119.0,952.5,2018-03-01 02:00:00
3,2018-03-01T03:00,34.4,0.0,0.0,0.0,0.0,1020.0,897.0,123.0,965.4,2018-03-01 03:00:00
4,2018-03-01T04:00,35.7,0.0,0.0,0.0,0.0,1010.0,888.0,122.0,963.7,2018-03-01 04:00:00


In [3]:
S3 = S3ROCManager(config['procdata_info'])

In [9]:
all_wells = S3.list_all_wells()

In [10]:
all_weather_stations = S3.list_weather_stations()

In [12]:
all_weather_stations

{'/Daralingie',
 '/Durham',
 '/Epsilon',
 '/Jackson',
 '/Moomba',
 '/Mount Margaret',
 '/Tirrawarra',
 '/Woomanooka',
 '/mount margaret',
 '/tirrawarra',
 '/woomanooka'}

In [16]:
with open(RepoPath.config("nearest_station.yaml"), 'r') as file:
    station_dict = yaml.safe_load(file)

In [30]:
required_stations = list(set(station_dict.values()))
required_stations

['Moomba', 'Tirrawarra', 'Epsilon', 'Woomanooka', 'Jackson']

In [31]:
for station in required_stations:
    station_df = S3.read_solar(station,'2016-01-01','2023-01-01')
    station_df.to_csv(RepoPath.data(f"{station}_weather.csv"), index = False)

Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160101_20160201.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160201_20160301.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160301_20160401.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160401_20160501.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160501_20160601.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160601_20160701.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160701_20160801.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160801_20160901.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20160901_20161001.csv doesn't exists on the provided S3 Bucket.
Object ROC/SOLAR_DATA/Moomba_SOLAR_DATA_20161001_20161101.csv do

In [27]:
station_df

,TS,temperature_2m,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,relativehumidity_2m,rain
0,2018-01-01 00:00:00,34.4,7.0,0.0,12.0,0.0,674.0,449.0,225.0,619.7,NaN,NaN
1,2018-01-01 01:00:00,36.0,46.0,0.0,73.0,8.0,763.0,408.0,355.0,473.2,NaN,NaN
2,2018-01-01 02:00:00,36.3,88.0,0.0,100.0,94.0,707.0,222.0,485.0,232.8,NaN,NaN
3,2018-01-01 03:00:00,35.9,89.0,0.0,98.0,99.0,441.0,100.0,341.0,100.7,NaN,NaN
4,2018-01-01 04:00:00,33.5,89.0,0.0,99.0,99.0,221.0,1.0,220.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
40531,2022-08-16 19:00:00,9.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0.0
40532,2022-08-16 20:00:00,8.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,0.0
40533,2022-08-16 21:00:00,8.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0,0.0
40534,2022-08-16 22:00:00,8.3,0.0,0.0,0.0,0.0,39.0,15.0,24.0,172.1,76.0,0.0


In [9]:
with open(RepoPath.config("well_labels.yaml"),'r') as file:
    label = yaml.safe_load(file)

In [12]:
S3 = S3ROCManager(config['procdata_info'])
S3.read_labelled_data("TIRRA80", "2018-01-01","2019-01-01")

,ROC_VOLTAGE,FLOW,PRESSURE_TH,Mask_ROC_VOLTAGE,Mask_FLOW,Mask_PRESSURE_TH,labels,temperature_2m,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance
2018-01-07,"[27.46213913, 27.35430908, 27.44135094, 27.445...","[11.20779991, 11.65649986, 11.63371391, 11.610...","[1479.23132324, 1482.46655273, 1483.92749023, ...","[1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,"[34.0, 35.6, 35.6, 34.4, 32.7, 31.9, 31.1, 30....","[11.0, 59.0, 80.0, 88.0, 89.0, 90.0, 89.0, 90....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19.0, 89.0, 98.0, 100.0, 99.0, 100.0, 98.0, 1...","[0.0, 18.0, 69.0, 94.0, 99.0, 99.0, 100.0, 100...","[608.0, 677.0, 579.0, 247.0, 210.0, 207.0, 215...","[321.0, 327.0, 177.0, 8.0, 0.0, 10.0, 13.0, 2....","[287.0, 350.0, 402.0, 239.0, 210.0, 197.0, 202...","[444.8, 380.1, 185.8, 8.1, 0.0, 11.0, 16.5, 3...."
2018-01-08,"[27.58503914, 27.56736946, 27.572349546666665,...","[9.60813999, 10.0741396, 17.38467026, 14.23071...","[1490.19018555, 1492.38146973, 1540.70092773, ...","[1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,"[30.5, 33.2, 34.2, 35.4, 36.2, 36.4, 36.6, 35....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[754.0, 932.0, 1053.0, 1106.0, 1088.0, 1001.0,...","[634.0, 799.0, 912.0, 962.0, 946.0, 865.0, 725...","[120.0, 133.0, 141.0, 144.0, 142.0, 136.0, 127...","[880.4, 929.9, 957.8, 969.2, 967.5, 951.9, 917..."
2018-01-09,"[27.60180092, 27.58165932, 27.58101082, 27.610...","[10.99120045, 10.6584301, 10.061335085, 9.4642...","[1448.44482422, 1436.02539063, 1432.79064941, ...","[1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, ...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,"[29.8, 31.4, 32.3, 33.4, 34.4, 34.5, 34.5, 34....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[769.0, 946.0, 1066.0, 1119.0, 1100.0, 1011.0,...","[647.0, 812.0, 925.0, 974.0, 957.0, 873.0, 732...","[122.0, 134.0, 141.0, 145.0, 143.0, 138.0, 127...","[900.4, 946.3, 972.2, 981.5, 978.6, 960.2, 925..."
2018-01-10,"[27.58452034, 27.60504913, 27.59271049, 27.591...","[10.492753343333334, 10.42055988, 10.52467012,...","[1467.75146484, 1471.92614746, 1472.44763184, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, ...","[0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,"[32.1, 34.1, 34.6, 35.5, 36.4, 36.9, 37.1, 37....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[769.0, 948.0, 1069.0, 1122.0, 1103.0, 1014.0,...","[648.0, 814.0, 927.0, 977.0, 959.0, 876.0, 734...","[121.0, 134.0, 142.0, 145.0, 144.0, 138.0, 128...","[903.7, 949.9, 975.0, 984.8, 980.5, 963.0, 927..."
2018-01-11,"[28.15733051, 28.15096092, 28.1687603, 28.1370...","[10.028419809999999, 10.01032988, 9.99223995, ...","[1458.98498535, 1455.54211426, 1456.06298828, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,"[37.0, 39.2, 40.3, 40.9, 41.6, 41.7, 42.0, 42....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[766.0, 9